In [12]:
############################################################ 손 지문영역 검출 코드
import cv2
import mediapipe as mp
import math

# MediaPipe Hands 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=10)

# 웹캠 초기화
cap = cv2.VideoCapture(0)

def calculate_distance(landmark1, landmark2, width, height):
    x1, y1 = int(landmark1.x * width), int(landmark1.y * height)
    x2, y2 = int(landmark2.x * width), int(landmark2.y * height)
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def draw_finger_circles(image, landmarks):
    h, w, _ = image.shape
    # 엄지손가락 끝과 새끼손가락 끝 사이의 거리 계산
    thumb_tip = landmarks[4]
    pinky_tip = landmarks[20]
    distance = calculate_distance(thumb_tip, pinky_tip, w, h)

    for i in range(4, 21, 4):  # 각 손가락의 끝부분 (지문이 있는 부분) 인덱스
        x = int(landmarks[i].x * w)
        y = int(landmarks[i].y * h)
        # 손가락 끝 부분에 빈 동그라미 그리기
        cv2.circle(image, (x, y), 10, (0, 0, 255), 2)  # 빨간색 빈 동그라미
    
    return image

while True:
    success, img = cap.read()
    if not success:
        break

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            img = draw_finger_circles(img, hand_landmarks.landmark)

    cv2.imshow("Hand Tracking", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

    


In [ ]:
############################################################ 손 블러링하는 코드
import cv2
import mediapipe as mp
import math

# MediaPipe Hands 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=10)

# 웹캠 초기화
cap = cv2.VideoCapture(0)

def calculate_distance(landmark1, landmark2, width, height):
    x1, y1 = int(landmark1.x * width), int(landmark1.y * height)
    x2, y2 = int(landmark2.x * width), int(landmark2.y * height)
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def blur_fingerprint_area(image, landmarks):
    h, w, _ = image.shape
    # 엄지손가락 끝과 새끼손가락 끝 사이의 거리 계산
    thumb_tip = landmarks[4]
    pinky_tip = landmarks[20]
    distance = calculate_distance(thumb_tip, pinky_tip, w, h)
    
    # 거리에 따라 블러 크기 조정
    blur_radius = int(max(15, min(50, distance // 4)))
    if blur_radius % 2 == 0:  # 홀수로 만들기
        blur_radius += 1
    ksize = (blur_radius, blur_radius)

    # 블러 처리 범위 조정
    blur_size = int(max(5, min(50, distance // 30)))
    
    # 화면에 거리와 ksize 표시
    # cv2.putText(image, f'Distance: {int(distance)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    # cv2.putText(image, f'ksize: {ksize}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    for i in range(4, 21, 4):  # 각 손가락의 끝부분 (지문이 있는 부분) 인덱스
        x = int(landmarks[i].x * w)
        y = int(landmarks[i].y * h)
        # 지문 부분 주변의 사각형 영역 설정
        x_start, y_start = max(0, x-blur_size), max(0, y-blur_size)
        x_end, y_end = min(w, x+blur_size), min(h, y+blur_size)
        # 블러 처리할 영역이 유효한지 확인
        if x_start < x_end and y_start < y_end:
            image[y_start:y_end, x_start:x_end] = cv2.GaussianBlur(image[y_start:y_end, x_start:x_end], ksize, 15)
    return image

while True:
    success, img = cap.read()
    if not success:
        break

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            img = blur_fingerprint_area(img, hand_landmarks.landmark)

    cv2.imshow("Hand Tracking", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
